# Energy Storage Project

In the next picture it is shown the configuration of the system as well as the different variables, as a first step we need to carry on an energy balance that must be satisfied. It is presented in the next equation:

<img width="400" src="system.jpg">


$\displaystyle \quad \theta _w[i] P_w+\theta_s[i] P_s + \alpha_d[i] P_d + \alpha_{nd}[i] P_{nd} = P_{dem}[i] + \alpha_b[i] P_b + P_{un} [i]$

Just as a note when the variables or constants have an index $[i]$, it represents that this variable correspond to the $i$ period. It will be one equation for each period.

An other important characteristic to model for the systems is the battery. It must be taken into account that the battery can be charged and discharged, changing the capacity. We can define a new variable $E_{max,b}$ which corresponds to the maximal energy that can be stored in the battery in MWh. Doing an energy balance for the battery: the energy stored in the $i+1$ period will be equal to the energy in the battery in the $i$ period plus the energy received or transferred from the battery, i.e.,

$\quad \displaystyle E_{b}[i+1] = E_{b}[i] + \alpha_{b}[i] P_b \Delta t$ 

Note that $-1 \leq \alpha_{b}[i] \leq 1$. When $\alpha_{b}[i]$ is positive it means that the baterry has been charged, when it is negative the batery has been discharged. To limit the charging and discharging process in function of the maximal capacity of the battery, the last equation is divided by $E_{max,b}$:

$\quad \displaystyle \frac{E_{b}[i+1]}{E_{max,b}} = \frac{E_b[i]}{E_{max,b}} + \frac{\alpha_{b}[i]P_b\Delta t}{E_{max,b}}$

It can be defined a new variable that is commonly founded in the literature when batteries are modelated, which is called State of Charge ($SOC$) defined as follows:

$\quad \displaystyle SOC[i]=\frac{E_b[i]}{E_{max,b}}$

also due to the fact that $\Delta t = 1 \text{ h}$ we can do not consider this term obtaining the next expression:

$\quad \displaystyle SOC[i+1]=SOC[i]+\alpha_b[i]\frac{P_b}{E_{max,b}}$

Then, it is needed to consider the next restriction:

$\quad \displaystyle 0 \leq SOC[i]\leq 1$

Next we present all the constant and variables used until now in the model proposed. 
##### Constants

| notation | meaning | units |
| -: | :- | :-: |
| $\theta_w[i]$ | nominal production of wind energy during the $i$ period. | - |
| $\theta_s[i]$ | nominal prodcution of solar energy during the $i$ period.| - |
| $P_w$ | maximal capacity of the wind plant.| MW |
| $P_s$ | maximal capacity of the solar plant.| MW |
| $P_d$ | maximal capacity of the old-diesel plant.| MW |
| $P_{dem}[i]$ | power demand during the $i$ period | MW |

##### Variables

| notation | meaning | units |
| -: | :- | :-: |
| $\alpha_d[i]$ | operating capacity factor of the old-diesel plant during the $i$ period | - |
| $\alpha_{nd}[i]$ | operating capacity factor of the new-diesel plant during the $i$ period | - |
| $P_{nd}$ | maximal capacity required of the new-diesel plant | MW |
| $\alpha_b[i]$ | operating capacity factor of the battery during the $i$ period | - |
| $P_{b}$ | maximal capacity required for the battery | MW |
| $P_{un}[i]$ | loss load during the $i$ period | MW |
| $SOC[i]$ | state of charge of the battery during the $i$ period | - |
| $E_{max,b}$ | maximal energy capacity of the battery | MWh |

To restring the factible set of the problem, there will be some restrictions in the variables, next we present all of thems:

$\quad 0 \leq \alpha_d[i] \leq 1$

$\quad 0 \leq \alpha_{nd}[i] \leq 1$

$\quad P_{nd} \geq 0$

$\quad-1 \leq \alpha_b[i] \leq 1$

$\quad P_b \geq 0$

$\quad P_{un}[i] \geq 0$

$\quad 0 \leq SOC[i] \leq 1$

$\quad E_{max,b} \geq 0$

Until now, just the system has been modeled. However, to carry on the optimization problem we need to define an objective function which depends on three factors: 1) the investment cost, 2) the operation and maintenance cost and 3) a variable cost. First, let's consider just the variable cost. Considering the different componentes has been taken into account, just a variable cost can be associated to the diesel generation and to the non-served energy:

$\displaystyle \quad C_{var} = \sum_{i=1} ^{N} \left[ \alpha_d[i]P_d C_d + \alpha_{nd}[i]P_{nd}C_d + P_{un}[i]C_{un}\right]  \Delta t$

where $N$ is the number of periods considered in the optimization period, $C_d$ is the unitary cost of diesel in €/MWh and $C_{un}$ the unitary cost of the unserved energy in €/MWh.

## First Model

As a first model, just the variable cost will be taken into consideration, this allows for the short-term simulations. The following lines show the code corresponding to the data loading process.

In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from datetime import datetime

data = pd.read_excel('2022-23_RES_island-project.xlsx', header = 3)
data = data.to_numpy()

def convert_date(date):
    date = date.split('-')
    w = date[2].split('T')
    date[2] = w[0]
    date.append(w[1].split(':')[0])
    date = [int(i) for i in date]
    date = datetime(date[0],date[1],date[2],date[3])
    return date

for item in data:
    item[0] = convert_date(item[0])
    
t = data[:,0]
P_dem = data[:,1]
theta_s = data[:,2]
theta_w = data[:,3]
P_w = 18
P_s = 25
P_d = 16
E_max = 2000

Cd = 508
Cun = 10000 

N = 48 #number of periods

## Variables defition

In [2]:
from docplex.mp.model import Model
mdl = Model('storage_team')

In [3]:
periods = [i for i in range(N)]

a_d = mdl.continuous_var_list(periods, name = 'ad')
a_nd = mdl.continuous_var_list(periods, name = 'a_nd')
a_b = mdl.continuous_var_list(periods, name = 'a_b')
P_un = mdl.continuous_var_list(periods, name = 'P_un')
SOC = mdl.continuous_var_list(periods, name = 'SOC')
P_nd = mdl.continuous_var(name = 'P_nd')
P_b = mdl.continuous_var(name = 'P_b')
#E_max = mdl.continuous_var(name = 'E_max')



### Objective Funciton

In [4]:
mdl.minimize(mdl.sum(a_d[i]*P_d*Cd + a_nd[i]*P_nd*Cd + P_un[i]*Cun for i in periods))

###  Restrictions

In [5]:
for i in periods[0:N-1]:
    mdl.add_constraint(E_max*SOC[i+1] == E_max*SOC[i] + a_b[i]*P_b)
    
for i in periods:
    mdl.add_constraint(a_d[i] <= 1)
    mdl.add_constraint(a_d[i] >= 0)
    mdl.add_constraint(a_nd[i] <= 1)
    mdl.add_constraint(a_nd[i] >= 0)
    mdl.add_constraint(a_b[i] <= 1)
    mdl.add_constraint(a_b[i] >= -1)
    mdl.add_constraint(SOC[i] <= 1)
    mdl.add_constraint(SOC[i] >= 0)
    mdl.add_constraint(P_un[i] >= 0)
    
    
mdl.add_constraint(SOC[0] == 0.5)
mdl.add_constraint(P_nd >= 0)
mdl.add_constraint(P_b >= 0)
mdl.add_constraint(E_max >= 0)

In [6]:
print(mdl.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: storage_team

Minimize
 obj: 8128 ad_0 + 8128 ad_1 + 8128 ad_2 + 8128 ad_3 + 8128 ad_4 + 8128 ad_5
      + 8128 ad_6 + 8128 ad_7 + 8128 ad_8 + 8128 ad_9 + 8128 ad_10 + 8128 ad_11
      + 8128 ad_12 + 8128 ad_13 + 8128 ad_14 + 8128 ad_15 + 8128 ad_16
      + 8128 ad_17 + 8128 ad_18 + 8128 ad_19 + 8128 ad_20 + 8128 ad_21
      + 8128 ad_22 + 8128 ad_23 + 8128 ad_24 + 8128 ad_25 + 8128 ad_26
      + 8128 ad_27 + 8128 ad_28 + 8128 ad_29 + 8128 ad_30 + 8128 ad_31
      + 8128 ad_32 + 8128 ad_33 + 8128 ad_34 + 8128 ad_35 + 8128 ad_36
      + 8128 ad_37 + 8128 ad_38 + 8128 ad_39 + 8128 ad_40 + 8128 ad_41
      + 8128 ad_42 + 8128 ad_43 + 8128 ad_44 + 8128 ad_45 + 8128 ad_46
      + 8128 ad_47 + 10000 P_un_0 + 10000 P_un_1 + 10000 P_un_2 + 10000 P_un_3
      + 10000 P_un_4 + 10000 P_un_5 + 10000 P_un_6 + 10000 P_un_7 + 10000 P_un_8
      + 10000 P_un_9 + 10000 P_un_10 + 10000 P_un_11 + 10000 P_un_12
      + 10000 P_

In [8]:
solucion = mdl.solve(log_output = True)

Version identifier: 22.1.1.0 | 2022-11-26 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPLEX Error  5002: 'q1' is not convex.
Presolve time = 0.00 sec. (0.02 ticks)
Error: Model has non-convex quadratic constraint, index=0
